In [22]:
import  time 
import gensim
import numpy as np
from encoder2decoder import build_model
from data_process import DataProcess
import conf
import warnings
warnings.filterwarnings('ignore')

In [23]:
t1=time.time()
dec_vec_model = gensim.models.Word2Vec.load(r'model/decoder_vector.m')
dec_useful_words = tuple(dec_vec_model.wv.vocab.keys())
t2=time.time()
print(t2-t1)

0.005686283111572266


In [52]:
de_embddings=[]
for w in dec_useful_words:
    de_embddings.append(dec_vec_model.wv[w])
decode_embddings=np.array(de_embddings)

In [53]:
#得到decode embdding的转至，便于normlize
decode_embdding=decode_embddings.T
#回复embdding状态
decode_embddings=((decode_embdding-np.mean(decode_embdding,axis=0))/(np.std(decode_embdding,axis=0))).T
#加入'__PAD__' embdding
decode_embddings=np.concatenate((np.zeros(conf.dec_embedding_length).reshape(1,conf.dec_embedding_length),decode_embddings),axis=0)

In [38]:
def load_model(model_path):

    model = build_model(training=False)

    model.load_weights(model_path)

    return model
def data_to_padding_ids(text_list):

    data_process = DataProcess(use_word2cut=True)
    enc_vocab = data_process.read_vocabulary(data_process.enc_vocab_file)

    enc_padding_ids_list = []

    for text in text_list:

        words = data_process.text_cut_object.cut([text.strip()])
        words_list = words[0].strip().split()

        enc_ids = [enc_vocab.get(word, data_process.__UNK__) for word in words_list]

        if len(enc_ids) > data_process.enc_input_length:
            enc_ids = enc_ids[:data_process.enc_input_length]

        enc_length = len(enc_ids)

        enc_padding_ids = []
        enc_padding_ids.extend([0] * (data_process.enc_input_length - enc_length))
        enc_padding_ids.extend([int(enc_ids[enc_length - l - 1]) for l in range(enc_length)])

        enc_padding_ids_list.append(np.array(enc_padding_ids))

    return np.array(enc_padding_ids_list)

In [39]:
 model = load_model("model/seq2seq_model_weights.h5")

In [55]:
t1=time.time()
text =[u"你愿意嫁给我吗？"]
padding_ids = data_to_padding_ids(text)
prediction = model.predict_on_batch(padding_ids)
for i in range(0,10):
    sum_mse=np.sum(np.square((prediction[0][i]-decode_embddings)),axis=1)
    min_index=np.argmin(sum_mse)
    if min_index==0:
        word="__PAD__"
    else:
        word=dec_useful_words[np.argmin(sum_mse)-1]
    print(word)
t2=time.time()
print('预测用时{} %s'.format(t2-t1))

__GO__
我
愿意
__EOS__
__PAD__
__PAD__
__PAD__
__PAD__
__PAD__
__PAD__
预测用时3.694567918777466 %s
